In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load my cleaned up data
books = pd.read_csv('data/books_step3_output.csv', index_col='book_id')

In [3]:
reviews = pd.read_csv('data/reviews_step3_output.csv', index_col='review_id')

In [4]:
books.head()

,text_reviews_count,average_rating,similar_books,description,authors,num_pages,publication_year,ratings_count,title,count_similar_books
book_id,,,,,,,,,,
13571772,5,4.28,"['13590139', '105963', '207585', '10503130', '...",The questions plaguing Captain America's dream...,['37450'],146,2012,51,Captain America: Winter Soldier (The Ultimate ...,161
35452242,1,4.05,[],The fight for Jason Delgado's life and soul be...,"['16209952', '853385']",160,2012,6,Bounty Hunter 4/3: My Life in Combat from Mari...,2
707611,6,4.06,[],These are the stories that catapulted Superman...,"['81563', '89537']",272,1997,51,"Superman Archives, Vol. 2",2
27036536,9,4.15,[],PRODUCT DESCRIPTION: The first new volume of G...,"['14965', '3188368', '131836', '7507599']",224,2016,39,"War Stories, Volume 3",2
11827783,26,3.16,"['6651573', '8835785', '15060759', '9466897', ...",Sir Arthur Conan Doyle's dauntless detective r...,"['61231', '4980321']",144,2011,114,Sherlock Holmes: Year One,203


In [5]:
reviews.head()

,Unnamed: 0,user_id,book_id,rating,review_text,year
review_id,,,,,,
66b2ba840f9bd36d6d27f46136fe4772,0,dc3763cdb9b2cae805882878eebb6a32,18471619,3,Sherlock Holmes and the Vampires of London \n ...,2013
72f1229aba5a88f9e72f0dcdc007dd22,1,bafc2d50014200cda7cb2b6acd60cd73,6315584,4,"I've never really liked Spider-Man. I am, howe...",2016
a75309355f8662caaa5e2c92ab693d3f,2,bafc2d50014200cda7cb2b6acd60cd73,29847729,4,"A very quick introduction, this is coming out ...",2016
c3cc5a3e1d6b6c9cf1c044f306c8e752,3,bafc2d50014200cda7cb2b6acd60cd73,18454118,5,I've been waiting so long for this. I first st...,2014
cc444be37ab0a42bfb4dd818cb5edd10,4,bafc2d50014200cda7cb2b6acd60cd73,2239435,4,The only thing more entertaining than this boo...,2013


In [10]:
reviews.drop('Unnamed: 0', axis=1, inplace=True)
reviews.head()

KeyError: "['Unnamed: 0'] not found in axis"

In [9]:
len(reviews.user_id.unique())

59347

That's a lot of users for me to be using a categorical variable for user id. I need to think through a bit more how I'm going to set up this data and what I'm looking to predict.

I'm looking to predict whether or not a user would assign a '5' rating to a given book.

# Feature Dataset Creation

My target variable is whether or not a user rates a specific book as a 5 or not. A 5 rating would be considered success, otherwise not. (Ideally we are recommending things readers will love, not just like somewhat)

In [11]:
# Combine dataframes into one data set where each observation is a review
results = pd.merge(left=reviews, right=books, on='book_id')

In [14]:
display(results.head())
display(results.info())

,user_id,book_id,rating,review_text,year,text_reviews_count,average_rating,similar_books,description,authors,num_pages,publication_year,ratings_count,title,count_similar_books
0,dc3763cdb9b2cae805882878eebb6a32,18471619,3,Sherlock Holmes and the Vampires of London \n ...,2013,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207
1,eaa54d876d841293059657fb80a9bba6,18471619,2,A graphic novel featuring Sherlock Holmes up a...,2014,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207
2,da7a0c5ee0c89973224d8853445be68e,18471619,3,12/31/13 Not as terrible as you might think. F...,2013,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207
3,41b1c110d428bbc49481036e896c0a6f,18471619,2,Accepting as fact vampires are real with no ba...,2015,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207
4,d7310760f68365d3ca747fa8b9310518,18471619,2,This graphic novel has Sherlock Holmes taking ...,2014,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207


<class 'pandas.core.frame.DataFrame'>
Int64Index: 385752 entries, 0 to 385751
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              385752 non-null  object 
 1   book_id              385752 non-null  int64  
 2   rating               385752 non-null  int64  
 3   review_text          385560 non-null  object 
 4   year                 385752 non-null  int64  
 5   text_reviews_count   385752 non-null  int64  
 6   average_rating       385752 non-null  float64
 7   similar_books        385752 non-null  object 
 8   description          370794 non-null  object 
 9   authors              385752 non-null  object 
 10  num_pages            385752 non-null  int64  
 11  publication_year     385752 non-null  int64  
 12  ratings_count        385752 non-null  int64  
 13  title                385752 non-null  object 
 14  count_similar_books  385752 non-null  int64  
dtypes: float64(1), in

None

In [16]:
# This will be my target variable: 5-star review
results['5star'] = [1 if x == 5 else 0 for x in results['rating']]
results[['rating','5star']].sample(25)

,rating,5star
256432,5,1
335745,5,1
83059,3,0
186693,0,0
356213,4,0
113898,4,0
251830,4,0
379696,3,0
39694,4,0
66929,4,0


In [17]:
results.head()

,user_id,book_id,rating,review_text,year,text_reviews_count,average_rating,similar_books,description,authors,num_pages,publication_year,ratings_count,title,count_similar_books,5star
0,dc3763cdb9b2cae805882878eebb6a32,18471619,3,Sherlock Holmes and the Vampires of London \n ...,2013,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207,0
1,eaa54d876d841293059657fb80a9bba6,18471619,2,A graphic novel featuring Sherlock Holmes up a...,2014,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207,0
2,da7a0c5ee0c89973224d8853445be68e,18471619,3,12/31/13 Not as terrible as you might think. F...,2013,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207,0
3,41b1c110d428bbc49481036e896c0a6f,18471619,2,Accepting as fact vampires are real with no ba...,2015,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207,0
4,d7310760f68365d3ca747fa8b9310518,18471619,2,This graphic novel has Sherlock Holmes taking ...,2014,50,3.26,"['10638896', '18124619', '2014491', '4523963',...",Sherlock Holmes died fighting Professor Moriar...,"['1152914', '6851837', '4027758']",93,2014,181,Sherlock Holmes and the Vampires of London (Sh...,207,0


In [18]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385752 entries, 0 to 385751
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              385752 non-null  object 
 1   book_id              385752 non-null  int64  
 2   rating               385752 non-null  int64  
 3   review_text          385560 non-null  object 
 4   year                 385752 non-null  int64  
 5   text_reviews_count   385752 non-null  int64  
 6   average_rating       385752 non-null  float64
 7   similar_books        385752 non-null  object 
 8   description          370794 non-null  object 
 9   authors              385752 non-null  object 
 10  num_pages            385752 non-null  int64  
 11  publication_year     385752 non-null  int64  
 12  ratings_count        385752 non-null  int64  
 13  title                385752 non-null  object 
 14  count_similar_books  385752 non-null  int64  
 15  5star            

In [24]:
# Create categorical variable for similiar books
results_dummies = pd.get_dummies(results['similar_books'])

In [26]:
# Declare explanatory variable
X = results.drop(['review_text','description','authors','title', 
                  'similar_books','publication_year', '5star'], axis=1).

In [27]:
# Declare target variable
y = results['5star']

In [28]:
# Create test_train split
from sklearn.model_selection import train_test_split
y=y.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [29]:
print(X_train.shape)
print(X_test.shape)
print(X.shape)

(289314, 9)
(96438, 9)
(385752, 9)


# Modeling

Ready for the first "draft" of my model